In [2]:
# Setup packages and file paths 
import os
from itertools import chain
from collections import Counter
import collections 
import requests
import json
import pandas as pd
import zipfile
from io import BytesIO

path = "/home/teijehidde/Documents/Git Blog and Coding/data_dump/"
data_file = "data_new2.json"

In [7]:
def downloadWikiNetworks(node_title, original_lang = "en", additional_langs = []): # "ar" "de", "fr", "nl"

    api_endpoint = "https://" + original_lang + ".wikipedia.org/w/api.php" # fr.wikipedia.org; https://en.wikipedia.org
    wiki_data = []
    
    # step 1: download data on the links of node_title wikipage.
    S = requests.Session()
    params_network_title = {
        "action": "query",
        "generator": "links",
        "titles": node_title,
        "gplnamespace": 0, 
        "gpllimit": 500, 
        "plnamespace": 0,
        "pllimit": 500, 
        "prop": "info|links", 
        "format": "json"
    }
    response = S.get(url=api_endpoint, params=params_network_title)
    wiki_data.append(response.json())

    while 'continue' in wiki_data[-1].keys():
        params_cont = params_network_title

        # the below has to be added if langlinks added.
        # if 'llcontinue' in wiki_data[-1]['continue']:
        #    params_cont["llcontinue"] = wiki_data[-1]['continue']['llcontinue'] 
        #    print('llcontinue: ' + params_cont["llcontinue"])

        if 'plcontinue' in wiki_data[-1]['continue']:
            params_cont["plcontinue"] = wiki_data[-1]['continue']['plcontinue'] 
            print('plcontinue: ' + params_cont["plcontinue"])

        elif 'gplcontinue' in wiki_data[-1]['continue']: 
            params_cont["gplcontinue"] = wiki_data[-1]['continue']['gplcontinue']
            print('gplcontinue: ' + params_cont["gplcontinue"])

        else: 
            break  

        response = S.get(url=api_endpoint, params = params_cont)
        wiki_data.append(response.json())

    # step 2: download data on the node_title wikipage.
    S = requests.Session()
    params_node_title = {
        "action": "query",
        "titles": node_title,
        "prop": "info|links|langlinks", 
        "plnamespace": 0, 
        "pllimit": 500,
        "lllimit": 500, 
        "format": "json"
    }
    response = S.get(url=api_endpoint, params=params_node_title)
    wiki_data.append(response.json())
    
    while 'continue' in wiki_data[-1].keys():
        params_cont = params_node_title
        params_cont["plcontinue"] = wiki_data[-1]['continue']['plcontinue']
        print('continue: ' + params_cont["plcontinue"])

        response = S.get(url=api_endpoint, params=params_cont)
        wiki_data.append(response.json())

    # Step 3: creating pandas data table and saving it as json file.
    list_new_wiki_data = [pd.DataFrame.from_dict(item['query']['pages']).transpose() for item in wiki_data]
    df_temp_wiki_data = pd.concat(list_new_wiki_data)
    df_saved_wiki_data = pd.read_json((path + data_file), orient='records')
    
    link_titles = [i['title'] for i in df_temp_wiki_data.loc[df_temp_wiki_data['title'] == node_title]['links'][0]] + [node_title]
    link_ids = []
    for title in link_titles: 
        link_ids.append(str(df_temp_wiki_data.loc[df_temp_wiki_data['title'] == title]['pageid'][0]))
    
    column_names = ['ns', 'title', 'missing', 'contentmodel', 'pagelanguage', 'pagelanguagehtmlcode', 'pageid', 'lastrevid', 'length', 'links', 'langlinks'] # check list_new_wiki_data for additional items. 
    index_numbers = link_ids
    df_new_wiki_data = pd.DataFrame(columns = column_names, index=index_numbers)

    for item in list_new_wiki_data: 
        df_new_wiki_data.update(item) #, errors = 'raise')
    df_saved_wiki_data = df_saved_wiki_data.append(df_new_wiki_data, ignore_index=True, sort=False)

    try: 
        df_saved_wiki_data.to_json((path + data_file), orient='records')
        print("Data succesfully saved.")

    except: 
        print("Something went wrong. Check code.")

    # for debugging only. 
    return wiki_data

In [9]:
# Kubernetes, Vaccine, Oxford 
# wiki_data = downloadWikiNetworks('Kubernetes')

In [5]:
# json_wiki_data = pd.read_json((path + data_file), orient='records')

In [10]:
# json_wiki_data

In [18]:
list_wiki_data = [pd.DataFrame.from_dict(item['query']['pages']).transpose() for item in wiki_data]

In [46]:


for i in range(len(list_wiki_data)): 
    compare_df = list_wiki_data[2].compare(list_wiki_data[3]) 




In [61]:
compare_df

links  \
                                                    self   
61118  [{'ns': 0, 'title': '10 Gigabit Ethernet'}, {'...   
78768                                                NaN   
65184  [{'ns': 0, 'title': '386BSD'}, {'ns': 0, 'titl...   
26386  [{'ns': 0, 'title': 'GNU General Public Licens...   
97368                                                NaN   

                                                          
                                                   other  
61118                                                NaN  
78768  [{'ns': 0, 'title': 'Acceptable use policy'}, ...  
65184  [{'ns': 0, 'title': 'IOS'}, {'ns': 0, 'title':...  
26386                                                NaN  
97368  [{'ns': 0, 'title': '7/9'}, {'ns': 0, 'title':...

In [70]:
test = {} 

for index, row in compare_df.iterrows():

    print 

    test[index] = [compare_df.loc[index, 'links']['self']] + [compare_df.loc[index, 'links']['other']]
    # 

In [71]:
test['78768']

[nan,
 [{'ns': 0, 'title': 'Acceptable use policy'},
  {'ns': 0, 'title': 'Acknowledgement (data networks)'},
  {'ns': 0, 'title': 'Anonymizer'},
  {'ns': 0, 'title': 'Apache HTTP Server'},
  {'ns': 0, 'title': 'Application firewall'},
  {'ns': 0, 'title': 'Application layer'},
  {'ns': 0, 'title': 'ArXiv (identifier)'},
  {'ns': 0, 'title': 'Ari Luotonen'},
  {'ns': 0, 'title': 'Authentication'},
  {'ns': 0, 'title': 'Authentication protocol'},
  {'ns': 0, 'title': 'Bandwidth (computers)'},
  {'ns': 0, 'title': 'Blacklist (Computing)'},
  {'ns': 0, 'title': 'Cache (computing)'},
  {'ns': 0, 'title': 'Captive portal'},
  {'ns': 0, 'title': 'Certificate authority'},
  {'ns': 0, 'title': 'Client (computing)'},
  {'ns': 0, 'title': 'Common Gateway Interface'},
  {'ns': 0, 'title': 'Computer networking'},
  {'ns': 0, 'title': 'Computer security'},
  {'ns': 0, 'title': 'Computer surveillance'},
  {'ns': 0, 'title': 'Content-control software'},
  {'ns': 0, 'title': 'Content filtering'},
  {'

In [16]:
test 

,ns,title,missing,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,pageid,touched,lastrevid,length,redirect,new,links
-1,0,Amazon EC2 Container Service,,wikitext,en,en,ltr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2,0,DOSBox-X,,wikitext,en,en,ltr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-3,0,Docker Inc.,,wikitext,en,en,ltr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-4,0,Nomad (software),,wikitext,en,en,ltr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5,0,QNX Hypervisor,,wikitext,en,en,ltr,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590852,0,Xen,NaN,wikitext,en,en,ltr,590852,2021-08-03T02:40:59Z,1036851902,42982,NaN,NaN,NaN
23013349,0,XenServer,NaN,wikitext,en,en,ltr,23013349,2021-08-03T02:40:59Z,765414620,44,,NaN,NaN
21200649,0,XtratuM,NaN,wikitext,en,en,ltr,21200649,2021-08-04T09:44:07Z,1030321325,5293,NaN,NaN,NaN
1266680,0,Z/VM,NaN,wikitext,en,en,ltr,1266680,2021-08-02T22:45:47Z,1030410825,2710,NaN,NaN,NaN


In [15]:
# test.iloc[10:80,]